<a href="https://colab.research.google.com/github/NandakumarGunalan/MLLaboratory/blob/main/Lab4_feature_eng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Preprocessing and Engieneering

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from datetime import timedelta
import datetime as dt
import matplotlib.pyplot as plt

from pathlib import Path
import time

In [ ]:
plt.rcParams['figure.figsize'] = [13, 5]

The representation of your data can have a bigger influence in the performance of your model than the type of model or the exact hyperparameters you use.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Pre-processing

Download the data: <br>
https://www.kaggle.com/c/avazu-ctr-prediction

In [ ]:
start = time.time()
data = pd.read_csv("/content/drive/MyDrive/MLLab/avazu_train_subset.csv")
print("time in minutes {}".format((time.time() - start)/60))

time in minutes 0.3880092740058899


**This** data took a while to load. Let's investigate why.

In [ ]:
data.shape

(2021449, 24)

In [ ]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2021449 entries, 0 to 2021448
Data columns (total 24 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   Unnamed: 0        int64 
 1   hour              int64 
 2   C1                int64 
 3   banner_pos        int64 
 4   site_id           object
 5   site_domain       object
 6   site_category     object
 7   app_id            object
 8   app_domain        object
 9   app_category      object
 10  device_id         object
 11  device_ip         object
 12  device_model      object
 13  device_type       int64 
 14  device_conn_type  int64 
 15  C14               int64 
 16  C15               int64 
 17  C16               int64 
 18  C17               int64 
 19  C18               int64 
 20  C19               int64 
 21  C20               int64 
 22  C21               int64 
 23  click             int64 
dtypes: int64(15), object(9)
memory usage: 1.2 GB


The raw training data is 291.7 MB but pandas is using 1.2 GB. One of the reasons is that pandas is using int64 and object types.

To reduce the memory usage: <br>
1. We can use smaller types for numerical columns. To select the appropiate type read here
https://docs.scipy.org/doc/numpy-1.13.0/user/basics.types.html. For example int8 is (-128 to 127).
2. Covert categorical variables into categorical type
3. You can drop columns that are not important (for example id)

In [ ]:
data.head()

,Unnamed: 0,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,click
0,6442286,14102210,1005,0,5bcf81a2,9d54950b,f028772b,ecad2386,7801e8d9,07d7df22,...,2,21770,320,50,2507,0,35,34640,157,0
1,7691452,14102213,1005,0,85f751fd,c4e18dd6,50e219e0,e9739828,df32afa9,cef3e649,...,0,21768,320,50,2506,0,35,65535,157,0
2,23736315,14102622,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,0,15706,320,50,1722,0,35,34548,79,0
3,33772713,14102908,1005,1,85f751fd,c4e18dd6,50e219e0,1dc72b4d,2347f47a,0f2161f8,...,0,23647,300,50,2709,3,35,65535,23,0
4,12732389,14102318,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,0,15705,320,50,1722,0,35,34548,79,0


In [ ]:
data.describe() # look at min and max of every colunm. Can we change column types?

,Unnamed: 0,hour,C1,banner_pos,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,click
count,2.021449e+06,2.021449e+06,2.021449e+06,2.021449e+06,2.021449e+06,2.021449e+06,2.021449e+06,2.021449e+06,2.021449e+06,2.021449e+06,2.021449e+06,2.021449e+06,2.021449e+06,2.021449e+06,2.021449e+06
mean,2.021442e+07,1.410256e+07,1.004968e+03,2.877357e-01,1.015313e+00,3.319802e-01,1.884283e+04,3.188823e+02,6.008309e+01,2.112642e+03,1.434276e+00,2.273034e+02,4.908411e+04,8.333633e+01,1.694987e-01
std,1.167083e+07,2.966840e+02,1.095282e+00,5.056284e-01,5.272141e-01,8.556300e-01,4.955432e+03,2.136994e+01,4.722697e+01,6.091073e+02,1.326377e+00,3.512474e+02,1.544879e+04,7.028119e+01,3.751919e-01
min,5.000000e+00,1.410210e+07,1.001000e+03,0.000000e+00,0.000000e+00,0.000000e+00,3.750000e+02,1.200000e+02,2.000000e+01,1.120000e+02,0.000000e+00,3.300000e+01,3.446400e+04,1.000000e+00,0.000000e+00
25%,1.010697e+07,1.410230e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,1.692000e+04,3.200000e+02,5.000000e+01,1.863000e+03,0.000000e+00,3.500000e+01,3.454800e+04,2.300000e+01,0.000000e+00
50%,2.021355e+07,1.410260e+07,1.005000e+03,0.000000e+00,1.000000e+00,0.000000e+00,2.034600e+04,3.200000e+02,5.000000e+01,2.323000e+03,2.000000e+00,3.900000e+01,3.465700e+04,6.100000e+01,0.000000e+00
75%,3.032134e+07,1.410281e+07,1.005000e+03,1.000000e+00,1.000000e+00,0.000000e+00,2.189400e+04,3.200000e+02,5.000000e+01,2.526000e+03,3.000000e+00,1.710000e+02,6.553500e+04,1.010000e+02,0.000000e+00
max,4.042896e+07,1.410302e+07,1.012000e+03,7.000000e+00,5.000000e+00,5.000000e+00,2.405200e+04,1.024000e+03,1.024000e+03,2.758000e+03,3.000000e+00,1.839000e+03,6.553500e+04,2.550000e+02,1.000000e+00


In [ ]:
data["device_conn_type"].value_counts()

,count
device_conn_type,
0,1743816
2,166180
3,109272
5,2181


In [ ]:
types = {
    'click': np.uint8,
    'hour': np.uint32,            # keep uint32 if you plan to parse hour later
    'C1': np.uint16,
    'banner_pos': np.uint8,
    'site_id': 'category',
    'site_domain': 'category',
    'site_category': 'category',
    'app_id': 'category',
    'app_domain': 'category',
    'app_category': 'category',
    'device_id': 'category',
    'device_ip': 'category',
    'device_model': 'category',
    'device_type': np.uint8,
    'device_conn_type': np.uint8,
    'C14': np.uint32,
    'C15': np.uint16,
    'C16': np.uint16,
    'C17': np.uint16,
    'C18': np.uint16,
    'C19': np.uint16,
    'C20': np.uint16,
    'C21': np.uint16
}


In [ ]:
start = time.time()

data= pd.read_csv(
    "/content/drive/MyDrive/MLLab/avazu_train_subset.csv",
    usecols=types.keys(),
    dtype=types
)

print(f"Loaded in {(time.time() - start)/60:.2f} minutes")
print(f"Memory usage: {data.memory_usage(deep=True).sum() / 1024**2:.2f} MB")


Loaded in 0.37 minutes
Memory usage: 202.24 MB


### Reducing memory usage,

In [ ]:
print(data.info(memory_usage='deep'))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2021449 entries, 0 to 2021448
Data columns (total 23 columns):
 #   Column            Dtype   
---  ------            -----   
 0   hour              uint32  
 1   C1                uint16  
 2   banner_pos        uint8   
 3   site_id           category
 4   site_domain       category
 5   site_category     category
 6   app_id            category
 7   app_domain        category
 8   app_category      category
 9   device_id         category
 10  device_ip         category
 11  device_model      category
 12  device_type       uint8   
 13  device_conn_type  uint8   
 14  C14               uint32  
 15  C15               uint16  
 16  C16               uint16  
 17  C17               uint16  
 18  C18               uint16  
 19  C19               uint16  
 20  C20               uint16  
 21  C21               uint16  
 22  click             uint8   
dtypes: category(9), uint16(8), uint32(2), uint8(4)
memory usage: 202.2 MB
None


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/MLLab/avazu_train_subset.csv")

In [ ]:
#Rename the column called Unnamed: 0 to id directly inside the existing DataFrame
data.rename(columns={'Unnamed: 0':'id'}, inplace=True)


In [ ]:
data.head()

,id,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,click
0,6442286,14102210,1005,0,5bcf81a2,9d54950b,f028772b,ecad2386,7801e8d9,07d7df22,...,2,21770,320,50,2507,0,35,34640,157,0
1,7691452,14102213,1005,0,85f751fd,c4e18dd6,50e219e0,e9739828,df32afa9,cef3e649,...,0,21768,320,50,2506,0,35,65535,157,0
2,23736315,14102622,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,0,15706,320,50,1722,0,35,34548,79,0
3,33772713,14102908,1005,1,85f751fd,c4e18dd6,50e219e0,1dc72b4d,2347f47a,0f2161f8,...,0,23647,300,50,2709,3,35,65535,23,0
4,12732389,14102318,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,0,15705,320,50,1722,0,35,34548,79,0


### Splitting dataset based on time

Here is one example in which we should split the data based on time.

This function splits the data by time (hour) so the model trains on earlier records and is tested on later ones.


In [ ]:
def split_based_hour(data):
    """ Split data based on column hour.

    Use 20% of the date for validation.
    Inputs:
       data: dataframe from avazu
    Returns:
       train:
       val_test: 30% of the largest values of column "hour".
    """
    N = int(0.7*len(data))
    data = data.sort_values(by="hour")
    train = data[:N].copy()
    val_test = data[N:].copy()
    val_test.reset_index(drop=True, inplace=True)
    val = val_test.sample(frac = 0.5)
    test = val_test.drop(val.index)
    return train.reset_index(drop=True), val.reset_index(drop=True), test.reset_index(drop=True)

In [ ]:
train, val, test = split_based_hour(data)

In [ ]:
train.shape, val.shape, test.shape

((1415014, 24), (303218, 24), (303217, 24))

In [ ]:
train.head()

,id,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,...,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21,click
0,36805,14102100,1005,0,72af592e,19944fab,3e814130,ecad2386,7801e8d9,07d7df22,...,0,20366,320,50,2333,0,39,34567,157,0
1,115834,14102100,1005,0,85f751fd,c4e18dd6,50e219e0,febd1138,82e27996,0f2161f8,...,0,21189,320,50,2424,1,161,34657,71,0
2,69441,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,...,0,15704,320,50,1722,0,35,65535,79,0
3,115307,14102100,1005,0,cd58172f,b9c4ab81,f028772b,ecad2386,7801e8d9,07d7df22,...,0,20596,320,50,2161,0,35,65535,157,0
4,59368,14102100,1005,0,85f751fd,c4e18dd6,50e219e0,98fed791,d9b5648e,0f2161f8,...,0,19743,320,50,2264,3,427,34466,61,1


This function removes the specified column (here, 'id') from all dataset splits — train, validation, and test.
The 'id' column usually doesn’t carry predictive value, so dropping it avoids noise.
It returns cleaner datasets ready for modeling.

In [ ]:
# delete id column
def drop_column(train, val, test, col):
    train = train.drop(columns=[col])
    val = val.drop(columns=[col])
    test = test.drop(columns=[col])
    return train, val, test

train, val, test = drop_column(train, val, test, 'id')

This function separates the target variable from the features.
It stores the target column (Y) as a NumPy array and the remaining columns (X) as the input features.
This is a standard step before training machine learning models.

In [ ]:
def split_target(df, target):
    Y = df[target].values
    X = df.drop(columns=[target])
    return X, Y

In [ ]:
counts = train.groupby(['site_category']).size().reset_index(name='counts')
counts

NameError: name 'train' is not defined

This function groups infrequent categories (those appearing fewer than min_count times) into a single label 'UNK'.
It helps reduce the number of unique categories, simplifying the model and preventing overfitting.
Both training and validation sets are updated consistently.

In [ ]:
def group_rare_categories(train, val, col, min_count=100):
    counts = train.groupby([col]).size().reset_index(name='counts')
    top_categories = counts[counts['counts'] > min_count]
    new_categories = top_categories[col].values
    train[col] = np.array([x if x in new_categories else 'UNK' for x in train[col].values])
    val[col] = np.array([x if x in new_categories else 'UNK' for x in val[col].values])
    return train, val

In [ ]:
train, val = group_rare_categories(train, val, 'site_category', min_count=100)

In [ ]:
freq_counts = train.groupby(['site_category']).size()

In [ ]:
freq_counts = freq_counts/freq_counts.sum()

In [ ]:
freq_counts

,0
site_category,
0569f928,0.000430
28905ebd,0.196075
335d28a8,0.003269
3e814130,0.082879
50e219e0,0.391475
70fb0e29,0.000422
72722551,0.000674
75fa27f6,0.002991
76b2941d,0.001999


## Feature Engineering: Frequency encoding

Each category is replaced by the frequency of that category in the training data. Used for tree-based methods.
```
["a", "a", "a", "b", "c"]
```
is encoded as
```
[3/5, 3/5, 3/5, 1/5, 1/5]
```

In [ ]:
train.shape

(1415014, 23)

That line performs frequency encoding.  it replaces each category in "site_category" with how often it appears in the training data.
freq_counts is a dictionary mapping each category to its occurrence count or proportion.
The result, train["site_category_freq"], is a numeric feature better suited for tree-based models.

In [ ]:
train["site_category_freq"] = train["site_category"].map(freq_counts)

In [ ]:
val["site_category_freq"] = val["site_category"].map(freq_counts)

In [ ]:
train.shape

(1415014, 24)

In [ ]:
# checking that there are not nas
val.site_category_freq.notna().sum(), val.shape

(np.int64(303218), (303218, 24))

## Date and time features

* Periodicity
    * Day number in week, month, season, year, second, minute, hour
* Time since
* Difference between dates

In [ ]:
train.hour.unique()

array([14102100, 14102101, 14102102, 14102103, 14102104, 14102105,
       14102106, 14102107, 14102108, 14102109, 14102110, 14102111,
       14102112, 14102113, 14102114, 14102115, 14102116, 14102117,
       14102118, 14102119, 14102120, 14102121, 14102122, 14102123,
       14102200, 14102201, 14102202, 14102203, 14102204, 14102205,
       14102206, 14102207, 14102208, 14102209, 14102210, 14102211,
       14102212, 14102213, 14102214, 14102215, 14102216, 14102217,
       14102218, 14102219, 14102220, 14102221, 14102222, 14102223,
       14102300, 14102301, 14102302, 14102303, 14102304, 14102305,
       14102306, 14102307, 14102308, 14102309, 14102310, 14102311,
       14102312, 14102313, 14102314, 14102315, 14102316, 14102317,
       14102318, 14102319, 14102320, 14102321, 14102322, 14102323,
       14102400, 14102401, 14102402, 14102403, 14102404, 14102405,
       14102406, 14102407, 14102408, 14102409, 14102410, 14102411,
       14102412, 14102413, 14102414, 14102415, 14102416, 14102

This extracts the hour (last two digits) from the hour column to create a numeric feature hour_enc.
It helps models learn time-of-day patterns that may affect click behavior.

In [ ]:
# hour: format is YYMMDDHH, so 14091123 means 23:00 on Sept. 11, 2014 UTC.
train['hour_enc'] = train['hour'].astype(str).str[-2:].astype(float)
val['hour_enc'] = val['hour'].astype(str).str[-2:].astype(float)
train.head()

,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,...,C15,C16,C17,C18,C19,C20,C21,click,site_category_freq,hour_enc
0,14102100,1005,0,5b08c53b,7687a86e,3e814130,ecad2386,7801e8d9,07d7df22,a99f214a,...,300,250,1994,2,39,65535,33,0,0.082879,0.0
1,14102100,1002,0,61a8c644,948ff336,50e219e0,ecad2386,7801e8d9,07d7df22,5e1cc472,...,320,50,2253,2,303,65535,52,0,0.391475,0.0
2,14102100,1005,0,030440fe,08ba7db9,76b2941d,ecad2386,7801e8d9,07d7df22,a99f214a,...,320,50,2161,0,35,65535,157,0,0.001999,0.0
3,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,...,320,50,1722,0,35,65535,79,0,0.196075,0.0
4,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,...,320,50,1722,0,35,65535,79,1,0.196075,0.0


In [ ]:
train.head()

,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,...,C15,C16,C17,C18,C19,C20,C21,click,site_category_freq,hour_enc
0,14102100,1005,0,5b08c53b,7687a86e,3e814130,ecad2386,7801e8d9,07d7df22,a99f214a,...,300,250,1994,2,39,65535,33,0,0.082879,0.0
1,14102100,1002,0,61a8c644,948ff336,50e219e0,ecad2386,7801e8d9,07d7df22,5e1cc472,...,320,50,2253,2,303,65535,52,0,0.391475,0.0
2,14102100,1005,0,030440fe,08ba7db9,76b2941d,ecad2386,7801e8d9,07d7df22,a99f214a,...,320,50,2161,0,35,65535,157,0,0.001999,0.0
3,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,...,320,50,1722,0,35,65535,79,0,0.196075,0.0
4,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,07d7df22,a99f214a,...,320,50,1722,0,35,65535,79,1,0.196075,0.0


## Feature Engineering: Target or mean encoding
Use the target varible to generate features

This sample DataFrame shows how to create features from the target variable, using state and its corresponding y values.

By grouping the data by state and computing the average of the target y.
Example:

In [ ]:
data = pd.DataFrame({
        'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
        'y': [1, 1, 0, 0, 0]})
data

,state,y
0,Ohio,1
1,Ohio,1
2,Ohio,0
3,Nevada,0
4,Nevada,0


In [ ]:
m = data["y"].groupby(data["state"]).mean()
m

,y
state,
Nevada,0.000000
Ohio,0.666667


This line applies target encoding by mapping each state to its precomputed mean target value m.
It creates a new numeric feature, state_mean_enc, that captures how likely each state is to have y = 1.

In [ ]:
data["state_mean_enc"] = data["state"].map(m)
data

,state,y,state_mean_enc
0,Ohio,1,0.666667
1,Ohio,1,0.666667
2,Ohio,0,0.666667
3,Nevada,0,0.000000
4,Nevada,0,0.000000


This calculates the average click rate for each site_category.
It groups the training data by site_category and computes the mean of the target variable click for each group.

In [ ]:
mean_site_cat = train["click"].groupby(train["site_category"]).mean()

In [ ]:
mean_site_cat

,click
site_category,
0569f928,0.054187
28905ebd,0.211841
335d28a8,0.096412
3e814130,0.283488
50e219e0,0.130281
70fb0e29,0.108878
72722551,0.064990
75fa27f6,0.151938
76b2941d,0.039958


This line assigns each site_category its corresponding average click rate from mean_site_cat.
It creates a new numeric feature, site_category_mean_enc, representing the likelihood of a click for that category.

In [ ]:
train["site_category_mean_enc"] = train["site_category"].map(mean_site_cat)

In [ ]:
val["site_category_mean_enc"] = val["site_category"].map(mean_site_cat)

Note that mean encodoing needs to be computed on train and joined latter with validation and test.

Regularized mean encoding for the homework.

## Aggregate by one or multiple categorical features

Here are some fetures that we can compute
* Number of times `device_ip` appears on the training data. It would be better if we had `user_id`.
* Number of times `device_ip` appears per month on the training data.
* Min, max, average `C1` per `site_id`.

We can compute these features using `groupby` to aggregate to a new dataframe and then use `merge` to make the new feature.

### Categorical encoding can be slow. Let's three options

Groupby aggregation → creates numeric summary features.


Hash encoding → fast conversion for very large categorical spaces.


Label encoding → compact and model-friendly numeric IDs.



In [ ]:
def hash_encode_cols(train, val, col):
    print("Encoding column ",col)
    train[col] = train[col].apply(hash).astype("int32")
    val[col] = val[col].apply(hash).astype("int32")

In [ ]:
def label_encoding_with_NAs(train, val, col):
    train[col] = train[col].astype('category').cat.as_ordered()
    encoder = train[col].cat.categories
    # leaving 0 for NAs
    train[col] = train[col].cat.codes + 1
    val[col] = pd.Categorical(val[col], categories=encoder, ordered=True)
    val[col] = val[col].cat.codes + 1

In [ ]:
t = pd.DataFrame({"x":["a", "b", "c"]})
v = pd.DataFrame({"x":["a", "b", "d", "f"]})

In [ ]:
t['x_enc'] = t['x'].astype('category').cat.as_ordered()
encoder = t['x_enc'].cat.categories

In [ ]:
v['x_enc'] = pd.Categorical(v['x'], categories=encoder, ordered=True)

In [ ]:
t['x_enc'] = t['x_enc'].cat.codes + 1
t

,x,x_enc
0,a,1
1,b,2
2,c,3


In [ ]:
v['x_enc'] = v['x_enc'].cat.codes + 1

In [ ]:
v

,x,x_enc
0,a,1
1,b,2
2,d,0
3,f,0


This function performs label encoding while handling unseen categories with an "UNK" token.
It first fits a LabelEncoder on all unique training values (plus "UNK" if enabled).
Training data is encoded directly, while validation values not seen during training are replaced by "UNK".
This prevents errors when new categories appear in validation or test sets.
However, the list comprehension for handling unseen values makes it slow on large datasets.

In [ ]:
from sklearn.preprocessing import LabelEncoder
def label_encoding_with_UNK(train, val, col, UNK=True):
    """ Returns a label encoding "UNK" values
    """
    le = LabelEncoder()
    uniq = np.unique(train[col])
    if UNK:
        uniq = np.concatenate((np.array(["UNK"]),uniq))
    le.fit(uniq)
    train[col] = le.transform(train[col])
    # this is very slow
    val_col = [x if x in le.classes_ else 'UNK' for x in val[col]]
    val[col] = le.transform(val_col)

In [ ]:
col = "device_model"
t1 = train[[col, "click"]].copy()
v1 = val[[col ,"click"]].copy()

In [ ]:
start = time.time()
label_encoding_with_UNK(t1, v1, col)
print((time.time()-start)/60)

1.2069895267486572


In [ ]:
t2 = train[[col,"click"]].copy()
v2 = val[[col,"click"]].copy()

In [ ]:
start = time.time()
hash_encode_cols(t2, v2, col)
print((time.time()-start)/60)

Encoding column  device_model
0.022806990146636962


In [ ]:
col = "device_model"
t3 = train[[col,"click"]].copy()
v3 = val[[col,"click"]].copy()
start = time.time()
label_encoding_with_NAs(t3, v3, col)
print((time.time()-start)/60)

0.007503573099772135


In [ ]:
t1.head()

,device_model,click
0,2712,0
1,2096,0
2,393,0
3,623,0
4,623,1


In [ ]:
t2.head()

,device_model,click
0,1708846941,0
1,-20327128,0
2,-2113048854,0
3,1426357787,0
4,1426357787,1


In [ ]:
t3.head()

,device_model,click
0,2713,0
1,2097,0
2,394,0
3,624,0
4,624,1


In [ ]:
col = "device_model"
hash_encode_cols(train, val, col)

Encoding column  device_model


## BABY Baseline random forest model

In [ ]:
features = ["site_category_freq", "hour_enc", "site_category_mean_enc", "device_model"]

In [ ]:
y_train = train.click.values
y_val = val.click.values

In [ ]:
x_train = train[features]
x_val = val[features]
x_train.head()

,site_category_freq,hour_enc,site_category_mean_enc,device_model
0,0.082879,0.0,0.283488,1708846941
1,0.391475,0.0,0.130281,-20327128
2,0.001999,0.0,0.039958,-2113048854
3,0.196075,0.0,0.211841,1426357787
4,0.196075,0.0,0.211841,1426357787


In [ ]:
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def get_random_values_hyperparameters():
    min_samples_leaf = np.random.randint(low=50, high=100, size=1)[0]
    max_features = np.random.uniform(0.1,0.6)
    max_depth = np.random.randint(low=6, high=20, size=1)[0]
    return min_samples_leaf, max_features, max_depth

In [ ]:
get_random_values_hyperparameters()

(np.int64(90), 0.13347843095782116, np.int64(16))

This code performs a basic random search for hyperparameters of a Random Forest model.
It randomly selects different values for min_samples_leaf, max_features, and max_depth in each iteration.
For every model, it trains on the training data, predicts probabilities, and then reports log loss and AUC for both train and validation sets.
The goal is to find the simplest model that achieves the best validation performance (high AUC, low loss).

In [ ]:
# you want to look for the simplest model with the best val AUC or val loss
for i in range(5):
    h1, h2, h3 = get_random_values_hyperparameters()
    rf = RandomForestClassifier(n_jobs = -1, n_estimators=20, min_samples_leaf=h1,
                                 max_features=h2, max_depth=h3, verbose=1)
    rf.fit(x_train, y_train)
    y_hat_train = rf.predict_proba(x_train)[:,1]
    y_hat_val = rf.predict_proba(x_val)[:,1]
    print("min_samples_leaf %d, max_features %.1f, max_depth %d" % (h1, h2, h3))
    print("   train loss %.2f, val loss %.2f" % (log_loss(y_train, y_hat_train),
          log_loss(y_val, y_hat_val)))
    print("   train auc %.2f, val auc %.2f" % (roc_auc_score(y_train, y_hat_train),
          roc_auc_score(y_val, y_hat_val)))

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   34.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    2.8s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    0.5s finished


min_samples_leaf 69, max_features 0.3, max_depth 12
   train loss 0.44, val loss 0.42
   train auc 0.64, val auc 0.62


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   28.7s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    0.4s finished


min_samples_leaf 71, max_features 0.2, max_depth 10
   train loss 0.45, val loss 0.42
   train auc 0.63, val auc 0.62


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   36.5s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    2.4s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    0.7s finished


min_samples_leaf 62, max_features 0.6, max_depth 19
   train loss 0.44, val loss 0.42
   train auc 0.67, val auc 0.62


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   19.0s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    0.9s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    0.2s finished


min_samples_leaf 60, max_features 0.5, max_depth 6
   train loss 0.45, val loss 0.43
   train auc 0.61, val auc 0.61


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:   21.6s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    1.2s finished
[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  20 out of  20 | elapsed:    0.3s finished


min_samples_leaf 55, max_features 0.5, max_depth 9
   train loss 0.45, val loss 0.42
   train auc 0.62, val auc 0.61
